In [ ]:
import re
import csv
import random
import pandas as pd
from tqdm import tqdm
from itertools import islice
from vllm import LLM, SamplingParams
from transformers import AutoTokenizer

SEED = 1337
random.seed(SEED)

INFO 02-18 16:55:40 __init__.py:190] Automatically detected platform cuda.


In [54]:
df = pd.read_csv("../../data/data_raw/tweets.csv", encoding='latin-1', names=["target", "ids", "date", "flag", "user", "text"], on_bad_lines='skip')
df.head()

,target,ids,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [55]:
df.isna().sum()

target    0
ids       0
date      0
flag      0
user      0
text      0
dtype: int64

In [60]:
def has_non_standard_chars(s):
    return bool(re.match(r'^[a-zA-Z0-9\s.,!?\'\"-]+$', s))

In [61]:
df = df[df["text"].apply(has_non_standard_chars)]
df

,target,ids,date,flag,user,text,text_length
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,111
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,47
6,0,1467811592,Mon Apr 06 22:20:03 PDT 2009,NO_QUERY,mybirch,Need a hug,11
10,0,1467812416,Mon Apr 06 22:20:16 PDT 2009,NO_QUERY,erinx3leannexo,spring break in plain city... it's snowing,43
11,0,1467812579,Mon Apr 06 22:20:17 PDT 2009,NO_QUERY,pardonlauren,I just re-pierced my ears,26
...,...,...,...,...,...,...,...
1599990,4,2193579249,Tue Jun 16 08:38:59 PDT 2009,NO_QUERY,razzberry5594,WOOOOO! Xbox is back,21
1599992,4,2193579434,Tue Jun 16 08:39:00 PDT 2009,NO_QUERY,sdancingsteph,ReCoVeRiNg FrOm ThE lOnG wEeKeNd,33
1599995,4,2193601966,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,AmandaMarie1028,Just woke up. Having no school is the best fee...,56
1599997,4,2193601991,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,bpbabe,Are you ready for your MoJo Makeover? Ask me f...,57


In [62]:
df["text_length"] = df["text"].str.len()

/tmp/ipykernel_2983/3394612354.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["text_length"] = df["text"].str.len()


In [67]:
df.sort_values("text_length", ascending=False).tail(20000)

,target,ids,date,flag,user,text,text_length
207156,0,1973411539,Sat May 30 11:05:46 PDT 2009,NO_QUERY,Ashleia21,My tummy hurts,15
207349,0,1973461529,Sat May 30 11:11:35 PDT 2009,NO_QUERY,leanwithit,It stings ouch,15
207360,0,1973463962,Sat May 30 11:11:52 PDT 2009,NO_QUERY,FionaDeelish,Missin' my boi,15
154810,0,1936027990,Wed May 27 07:19:28 PDT 2009,NO_QUERY,chantal94x,"making french,",15
1520737,4,2176233408,Mon Jun 15 03:17:59 PDT 2009,NO_QUERY,ozzeth,just got home.,15
...,...,...,...,...,...,...,...
1050599,4,1960762577,Fri May 29 08:28:02 PDT 2009,NO_QUERY,this_kid2008,hey,6
4233,0,1468771833,Tue Apr 07 03:39:06 PDT 2009,NO_QUERY,goldieloxx,just,6
1071086,4,1966267712,Fri May 29 17:26:49 PDT 2009,NO_QUERY,Jsstar0273,Yes,6
310554,0,2001120036,Mon Jun 01 23:54:07 PDT 2009,NO_QUERY,Loois9,Ugh,6


In [68]:
df = df[df["text_length"] >= 15]
df

,target,ids,date,flag,user,text,text_length
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,111
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,47
10,0,1467812416,Mon Apr 06 22:20:16 PDT 2009,NO_QUERY,erinx3leannexo,spring break in plain city... it's snowing,43
11,0,1467812579,Mon Apr 06 22:20:17 PDT 2009,NO_QUERY,pardonlauren,I just re-pierced my ears,26
16,0,1467812964,Mon Apr 06 22:20:22 PDT 2009,NO_QUERY,lovesongwriter,Hollis' death scene will hurt me severely to w...,93
...,...,...,...,...,...,...,...
1599990,4,2193579249,Tue Jun 16 08:38:59 PDT 2009,NO_QUERY,razzberry5594,WOOOOO! Xbox is back,21
1599992,4,2193579434,Tue Jun 16 08:39:00 PDT 2009,NO_QUERY,sdancingsteph,ReCoVeRiNg FrOm ThE lOnG wEeKeNd,33
1599995,4,2193601966,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,AmandaMarie1028,Just woke up. Having no school is the best fee...,56
1599997,4,2193601991,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,bpbabe,Are you ready for your MoJo Makeover? Ask me f...,57


In [69]:
df.duplicated(subset="text").sum()

np.int64(8413)

In [3]:
def batchify(iterable, batch_size):
    """Splits an iterable into smaller batches."""
    iterable = iter(iterable)
    while batch := list(islice(iterable, batch_size)):
        yield batch

def save_to_csv(path, prompts, responses, temperature, top_p, top_k):
    """Saves prompts, responses and sampling parameters to a CSV file."""
    with open(path, mode='a', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        for prompt, response in zip(prompts, responses):
            writer.writerow([prompt, response, temperature, top_p, top_k])

def generate_responses(model, prompts, sampling_params):
    """Generate a batch of outputs using vLLM with customizable sampling parameters."""
    outputs = model.chat(prompts, sampling_params=sampling_params, use_tqdm=False)
    
    return [sample.outputs[0].text.replace('"', '') for sample in outputs]

In [4]:
BASE_PROMPT = [{"role": "system", "content": "You are a helpful asistant for rewritting tweets. Based on provided tweet generate a similar one. MAKE SURE TO REPLAY ONLY WITH THE SIMILAR TWEET."},
                {"role": "user", "content": "Tweet: \n {tweet} \n"},
                {"role": "assistant", "content": "Similar tweet: \n"}]

In [5]:
prompts = [
    [
        BASE_PROMPT[0],  # The system message
        {"role": "user", "content": BASE_PROMPT[1]["content"].format(tweet=tweet)},  # Formatted user message
        BASE_PROMPT[2]  # The assistant message
    ]
    for tweet in df["text"].values
]

In [6]:
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B-Instruct")

In [7]:
lens = []
batch_size = 128
for prompts_batch in tqdm(batchify(prompts, batch_size), total=len(prompts) // batch_size):
    tokens = tokenizer.apply_chat_template(prompts_batch)
    lens.extend([len(token) for token in tokens])

100%|██████████| 12500/12500 [05:53<00:00, 35.41it/s]


In [8]:
too_large = [i for i, l in enumerate(lens) if l > 32_768]
too_large

[]

In [5]:
df.drop(["target", "ids", "date", "flag", "user"], axis=1, inplace=True)
df.to_csv("../../data/data_human/tweets.csv", index=False)

In [ ]:
sampling_params = [
    SamplingParams(temperature=0.0, top_p=1.0, top_k=-1, max_tokens=40_000, seed=SEED),  # Pure Greedy (fully deterministic)
    SamplingParams(temperature=0.2, top_p=1.0, top_k=-1, max_tokens=40_000, seed=SEED),  # Highly Deterministic
    SamplingParams(temperature=0.5, top_p=0.95, top_k=100, max_tokens=40_000, seed=SEED), # Mildly Deterministic but Flexible
    SamplingParams(temperature=0.7, top_p=0.9, top_k=50, max_tokens=40_000, seed=SEED),  # Balanced and Natural
    SamplingParams(temperature=0.9, top_p=0.8, top_k=40, max_tokens=40_000, seed=SEED),  # Slightly More Diverse but Coherent
    SamplingParams(temperature=1.0, top_p=0.95, top_k=30, max_tokens=40_000, seed=SEED), # Default Creative Mode
    SamplingParams(temperature=1.2, top_p=0.7, top_k=20, max_tokens=40_000, seed=SEED),  # Highly Creative
]

In [10]:
llms = ["meta-llama/Llama-3.2-1B-Instruct"]
batch_size = 8
base_path = "../../data/data_ai/tweets/tweets_"

In [11]:
for llm in llms:
    model = LLM(model=llm, dtype="half", max_model_len = 10_000)
    csv_path = f"{base_path}{llm.split('/')[-1]}.csv"


    # init csv file
    with open(csv_path, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(["prompt", "response", "temperature", "top_p", "top_k"])

    cnt = 0
    for prompts_batch in tqdm(batchify(prompts, batch_size), total=len(prompts) // batch_size):
        params = random.choice(sampling_params)
        responses = generate_responses(model, prompts_batch, params)
        save_to_csv(csv_path, prompts_batch, responses, params.temperature, params.top_p, params.top_k)
        cnt += 1
        if cnt > 2:
            break

WARNING 02-14 21:45:22 config.py:2386] Casting torch.bfloat16 to torch.float16.
INFO 02-14 21:45:33 config.py:542] This model supports multiple tasks: {'embed', 'classify', 'generate', 'score', 'reward'}. Defaulting to 'generate'.
INFO 02-14 21:45:33 llm_engine.py:234] Initializing a V0 LLM engine (v0.7.2) with config: model='meta-llama/Llama-3.2-1B-Instruct', speculative_config=None, tokenizer='meta-llama/Llama-3.2-1B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=10000, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='xgrammar'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=False, c

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


INFO 02-14 21:45:47 model_runner.py:1115] Loading model weights took 2.3185 GB
INFO 02-14 21:45:50 worker.py:267] Memory profiling takes 2.46 seconds
INFO 02-14 21:45:50 worker.py:267] the current vLLM instance can use total_gpu_memory (6.00GiB) x gpu_memory_utilization (0.90) = 5.40GiB
INFO 02-14 21:45:50 worker.py:267] model weights take 2.32GiB; non_torch_memory takes 0.04GiB; PyTorch activation peak memory takes 1.21GiB; the rest of the memory reserved for KV Cache is 1.84GiB.
INFO 02-14 21:45:51 executor_base.py:110] # CUDA blocks: 3761, # CPU blocks: 8192
INFO 02-14 21:45:51 executor_base.py:115] Maximum concurrency for 10000 tokens per request: 6.02x
INFO 02-14 21:45:59 model_runner.py:1434] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI. If out-of-memory error occurs during cudagraph capture, consider decreasing `gpu_memory_utiliz

Capturing CUDA graph shapes: 100%|██████████| 35/35 [00:46<00:00,  1.33s/it]

INFO 02-14 21:46:45 model_runner.py:1562] Graph capturing finished in 47 secs, took 0.12 GiB
INFO 02-14 21:46:45 llm_engine.py:431] init engine (profile, create kv cache, warmup model) took 58.19 seconds



  0%|          | 0/200000 [00:00<?, ?it/s]

INFO 02-14 21:46:46 chat_utils.py:332] Detected the chat template content format to be 'string'. You can set `--chat-template-content-format` to override this.


  0%|          | 2/200000 [00:03<93:46:33,  1.69s/it]


In [12]:
df = pd.read_csv("../../data/data_ai/tweets/tweets_Llama-3.2-1B-Instruct.csv")
df.head()

,prompt,response,temperature,top_p,top_k
0,"[{'role': 'system', 'content': 'You are a help...",@switchfoot just when you thought you knew you...,1.2,0.7,20
1,"[{'role': 'system', 'content': 'You are a help...","Ugh, can't even post a pic of my lunch on Inst...",1.2,0.7,20
2,"[{'role': 'system', 'content': 'You are a help...",Just pulled off an amazing 50% save in a divin...,1.2,0.7,20
3,"[{'role': 'system', 'content': 'You are a help...",My skin is on flames I'm feeling like I'm goi...,1.2,0.7,20
4,"[{'role': 'system', 'content': 'You are a help...","@nationwideclass, i'm here for the same reason...",1.2,0.7,20
